In [19]:
import os
import sys
import random
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score

import keras
import tensorflow as tf
import keras.layers as L
from keras import layers
import keras.backend as K
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.losses import binary_crossentropy
from keras.utils import pad_sequences
from keras.layers import Layer, Dropout, Dense, Input, Embedding, Bidirectional, LSTM, Concatenate

curr_dir = os.path.dirname(os.path.realpath('__file__'))
proj_dir = os.path.abspath(os.path.join(curr_dir, '..'))

sys.path.append(proj_dir)

from src.configuration import load_config

def seed_everything(seed=2023):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything()

cfg = load_config()
print(cfg)

{'data': {'inp': './../inp', 'out': './../out'}, 'logs': './../logs', 'hyps': {'n_split': 5, 'max_len': {'assamese': 256}, 'random_state': [42, 2023]}}


In [20]:
train_df = pd.read_csv(os.path.join(cfg['data']['inp'], 'assamese/train_A_AH_HASOC2023.csv'))
test_df = pd.read_csv(os.path.join(cfg['data']['inp'], 'assamese/test_A_AH_HASOC2023.csv'))
sub_df = pd.read_csv(os.path.join(cfg['data']['inp'], 'assamese/sample.csv'))

In [21]:
MAXLEN = 256
VOCAB_SIZE = 20000
EMBED_DIM = 128
BATCH_SIZE = 128
EPOCHS = 10
CLASSES = 2

TEXT_COL = "text"
TARGET_COL = "task_1"

In [22]:
print("Length of train: ", len(train_df))
print("Length of test: ", len(test_df))

tokenizer = Tokenizer(num_words=VOCAB_SIZE, filters='!"#$&(),.:;?@[\\]^_`{|}\t\n')
tokenizer.fit_on_texts(list(train_df[TEXT_COL]) + list(test_df[TEXT_COL]))
word_idx = tokenizer.word_index

X_train = tokenizer.texts_to_sequences(list(train_df[TEXT_COL]))
y_train = train_df[TARGET_COL].map({'HOF': 0, 'NOT': 1})

X_test = tokenizer.texts_to_sequences(list(test_df[TEXT_COL]))

X_train = pad_sequences(X_train, maxlen=MAXLEN)
X_test = pad_sequences(X_test, maxlen=MAXLEN)

Length of train:  4036
Length of test:  1009


In [23]:
class AttentionBlock(keras.Model):

    def __init__(self, units):
        super(AttentionBlock, self).__init__()
        self.W1 = Dense(units=units)
        self.W2 = Dense(units=units)
        self.V = Dense(1)

    def call(self, features, hidden):
        hidden_t = K.expand_dims(hidden, 1)
        # additive attention
        score = K.tanh(self.W1(features) + self.W2(hidden_t))

        attn_weights = K.softmax(self.V(score), axis=1)

        context = attn_weights * features
        context = tf.reduce_sum(context, axis=1)

        return context, attn_weights
        pass

    pass


def build_model(max_len, max_features, embed_size, attn_units=20, num_classes=4, rnn_cell_size=32):
    seq_inp = Input(shape=max_len, dtype="int32")
    embedded_seq = Embedding(max_features, embed_size)(seq_inp)
    lstm = Bidirectional(LSTM(
        rnn_cell_size, return_sequences=True
    ), name="bilstm_0")(embedded_seq)

    lstm, f_h, f_c, b_h, b_c = Bidirectional(LSTM(
        rnn_cell_size, return_sequences=True, return_state=True
    ), name="bilstm_1")(lstm)

    h_ = Concatenate()([f_h, b_h])
    c_ = Concatenate()([f_c, b_c])

    context, attn_weights = AttentionBlock(attn_units)(lstm, h_)

    fc_pre = Dense(16, activation="relu")(context)
    do = Dropout(0.5)(fc_pre)
    output = Dense(1, activation="sigmoid")(do)

    return keras.Model(inputs=seq_inp, outputs=output)
    pass

In [24]:
model = build_model(max_len=MAXLEN, max_features=VOCAB_SIZE, embed_size=EMBED_DIM, num_classes=CLASSES)
model.compile(loss=tf.keras.losses.BinaryFocalCrossentropy(from_logits=False),
             optimizer="adam",
             metrics=[tf.keras.metrics.binary_crossentropy, tf.keras.metrics.binary_focal_crossentropy])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 256, 128)             2560000   ['input_1[0][0]']             
                                                                                                  
 bilstm_0 (Bidirectional)    (None, 256, 64)              41216     ['embedding[0][0]']           
                                                                                                  
 bilstm_1 (Bidirectional)    [(None, 256, 64),            24832     ['bilstm_0[0][0]']            
                              (None, 32),                                                     

In [25]:
cv_splits = list(StratifiedKFold(n_splits=5).split(X_train, y_train))

oof_preds = np.zeros((X_train.shape[0],))
test_preds = np.zeros((X_test.shape[0],))

for fold in range(5):
    K.clear_session()
    train_idx, val_idx = cv_splits[fold]
    print("\n======FOLD {}=====".format(fold))
    print()
    model.fit(X_train[train_idx], y_train[train_idx],
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(X_train[val_idx], y_train[val_idx]))

    oof_preds[val_idx] += model.predict(X_train[val_idx])[:, 0]
    test_preds += model.predict(X_test)[:, 0]


======FOLD 0=====

Epoch 1/10
26/26 [==============================] - 24s 644ms/step - loss: 0.1720 - binary_crossentropy: 0.6888 - binary_focal_crossentropy: 0.1720 - val_loss: 0.1704 - val_binary_crossentropy: 0.6861 - val_binary_focal_crossentropy: 0.1704
Epoch 2/10
26/26 [==============================] - 15s 576ms/step - loss: 0.1712 - binary_crossentropy: 0.6869 - binary_focal_crossentropy: 0.1712 - val_loss: 0.1705 - val_binary_crossentropy: 0.6868 - val_binary_focal_crossentropy: 0.1705
Epoch 3/10
26/26 [==============================] - 16s 636ms/step - loss: 0.1710 - binary_crossentropy: 0.6874 - binary_focal_crossentropy: 0.1710 - val_loss: 0.1704 - val_binary_crossentropy: 0.6865 - val_binary_focal_crossentropy: 0.1704
Epoch 4/10
26/26 [==============================] - 17s 635ms/step - loss: 0.1710 - binary_crossentropy: 0.6870 - binary_focal_crossentropy: 0.1710 - val_loss: 0.1703 - val_binary_crossentropy: 0.6855 - val_binary_focal_crossentropy: 0.1703
Epoch 5/10
26/26

In [26]:
print("Training Finished...")
print("Performance in training Data...")
oof_pred = tf.math.greater_equal(oof_preds, 0.5)
val_f1_score = f1_score(y_true=y_train, y_pred=oof_pred)
print("F1 Score for Training: ", val_f1_score)
print("Classification report for training: \n", classification_report(y_true=y_train, y_pred=oof_pred))

Training Finished...
Performance in training Data...
F1 Score for Training:  0.903169014084507
Classification report for training: 
               precision    recall  f1-score   support

           0       0.94      0.92      0.93      2347
           1       0.90      0.91      0.90      1689

    accuracy                           0.92      4036
   macro avg       0.92      0.92      0.92      4036
weighted avg       0.92      0.92      0.92      4036



In [28]:
y_preds = test_preds / 5

print("Evaluation on test data...")
y_pred = tf.math.greater_equal(y_preds, 0.5).numpy().astype(int)

Evaluation on test data...


In [29]:
test_df['task_1'] = y_pred
test_df['task_1'] = test_df['task_1'].map({0:'HOF', 1:'NOT'})
test_df['task_1'].value_counts()

task_1
HOF    584
NOT    425
Name: count, dtype: int64

In [33]:
test_df[['S. No.', 'task_1']].to_csv(f'./../out/baseline_lstm_submission_{val_f1_score}localF1.csv', index=False)